In [1]:
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
logging.basicConfig(
    level=logging.INFO,  # Set the minimum logging level to INFO
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

sys.path.append('../')
from PMpostprocess import PMpostprocess as pp
from PMpostprocess import signal_processing as sp

profilemonitor_plot_util start


In [2]:
import warnings
from contextlib import contextmanager

@contextmanager
def capture_warnings():
    """
    A context manager to capture and print warnings within a block of code.
    """
    with warnings.catch_warnings(record=True) as caught_warnings:
        warnings.simplefilter("always")  # Always capture warnings
        yield  # Allow code execution within the context
        for w in caught_warnings:
            print(f"Warning message: {w.message}")
            print(f"Warning type: {w.category.__name__}")
            print(f"Originated from: {w.filename}, line {w.lineno}")

In [3]:
sys.path.append('../../')
from FRIB_model import ISAAC_helper as ih

FRIB_model version: 1.0.0. updated on 2024-03-05


In [4]:
import os
ISAAC_database_path = "/home/devuser/sf_HWANG/Workspace/BPM4pickup/ISAAC_data_PMver0"
ISAAC_data_rel_path = "20240421_140218_maruta_FS1_CSS_PM_D2225_pv_scan_1st"
#"20240528_200129_hwang_FS1_CSS_PM_D2225_pv_scan"
isac_data = ih.get_most_recent_reconst_data(ISAAC_data_rel_path,ISAAC_database_path)

In [5]:
isac_data.keys()

dict_keys(['reconst_summary', 'reconst_input', 'reconst_output', 'fmlatfile'])

# summary data

In [6]:
isac_data['reconst_summary'].keys()

dict_keys(['comment', 'time', 'jsonfname', 'fname', 'pvinfo', 'scan_type', 'monitorl', 'putPVnamel', 'getPVnamel', 'initputPVvall', 'initgetPVvall', 'scan_data'])

In [7]:
isac_data['reconst_summary']['scan_data'][0].keys()

dict_keys(['putPVvall', 'getPVvall', 'res_monitorl'])

In [8]:
isac_data['reconst_summary']['scan_data'][0]['getPVvall']

{'FS1_CSS:PSQ_D2194:I_RD': 65.753, 'FS1_CSS:PSQ_D2202:I_RD': 104.281}

In [9]:
tmp = isac_data['reconst_summary']['scan_data'][0]['res_monitorl'][0]
tmp['name'], tmp['file'], tmp['coord'], tmp['xrms']

('FS1_CSS:PM_D2225',
 './/FS1_CSS_PM_D2225_20240421_140238.dat',
 'Suxy',
 1.9280020620522715)

# reconst input data

In [10]:
isac_data['reconst_input'].keys()

dict_keys(['meas', 'reconst_input', 'jsonfname'])

In [11]:
isac_data['reconst_input']['meas'][0].keys()

dict_keys(['flamevall', 'monitorl'])

In [12]:
isac_data['reconst_input']['reconst_input'].keys()

dict_keys(['model_engine', 'version', 'lattice_model', 'selem', 'eelem', 'flat', 'opt_method', 'iteration_no', 'comment', 'scan_type', 'monitorl', 'opt_flg', 'opt_target', 'measurement_select', 'moment_init_param', 'moment_weightl', 'moment_init_paraml', 'moment_weightl2', 'target_knobs_comment', 'target_knobs', 'monitor_knobs'])

In [13]:
assert isac_data['reconst_input']['reconst_input']['opt_target'][-1] == 'moment1'
isac_data['reconst_input']['reconst_input']['opt_target']

['moment1', 'moment1', 'moment1', 'moment1', 'moment1', 'moment1', 'moment1']

In [14]:
isac_data['reconst_input']['reconst_input']['measurement_select'][-1]['yrms']

[0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2]

In [15]:
isac_data['reconst_input']['reconst_input']['target_knobs_comment']

"((ename,[min_val, max_val, initial_val, field, unit]),('FE_LEBT:QHE_D0844',[-10, 10, 0, 'dy','mm']))"

# reconst output data

In [16]:
isac_data['reconst_output'].keys()

dict_keys(['meas', 'reconst_input', 'jsonfname', 'fjson_output', 'reconst_output'])

In [17]:
isac_data['reconst_output']['meas'][0].keys()

dict_keys(['flamevall', 'monitorl'])

In [18]:
isac_data['reconst_output']['reconst_input'].keys()

dict_keys(['model_engine', 'version', 'lattice_model', 'selem', 'eelem', 'flat', 'opt_method', 'iteration_no', 'comment', 'scan_type', 'monitorl', 'opt_flg', 'opt_target', 'measurement_select', 'moment_init_param', 'moment_weightl', 'moment_init_paraml', 'moment_weightl2', 'target_knobs_comment', 'target_knobs', 'monitor_knobs'])

In [19]:
isac_data['reconst_output']['reconst_output'].keys()

dict_keys(['posl', 'xdatal', 'xlabel', 'flat', 'pv_name', 'title', 'xcenl_meas', 'ycenl_meas', 'xcenl_sim', 'ycenl_sim', 'xrmsl_meas', 'yrmsl_meas', 'cxyl_meas', 'xrmsl_sim', 'yrmsl_sim', 'cxyl_sim', 'moment_init_param', 'moment_opt_param'])

In [20]:
isac_data['reconst_output']['reconst_output']['flat']

'/files/shared/ap/ISAAC/data/20240421_140218_maruta_FS1_CSS_PM_D2225_pv_scan/summary_20240421_140218_reconst_output_3d/flame_reconst_input.lat'

In [21]:
isac_data['reconst_output']['reconst_output']['moment_opt_param']

[0.25857486189020107,
 4.608323682757103,
 4.9106971747818395,
 0.11083098743154102,
 2.476651255551925,
 4.917041306489429,
 -2.1974874796585805e-06,
 -0.18290867391945426,
 0.04445012003205638,
 -0.07108212024379634]

# Read PM raw data

In [22]:
print(isac_data['reconst_input']['reconst_input']['measurement_select'][-1]['xrms'])
print(isac_data['reconst_input']['reconst_input']['measurement_select'][-1]['yrms'])

[2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2]


In [23]:
scan_data = isac_data['reconst_summary']['scan_data']
len(scan_data)

11

In [24]:
raw_data_flist = [scan_data[i]['res_monitorl'][0]['file'][3:] for i in range(len(scan_data))]
raw_data_flist

['FS1_CSS_PM_D2225_20240421_140238.dat',
 'FS1_CSS_PM_D2225_20240421_144741.dat',
 'FS1_CSS_PM_D2225_20240421_140447.dat',
 'FS1_CSS_PM_D2225_20240421_141827.dat',
 'FS1_CSS_PM_D2225_20240421_140656.dat',
 'FS1_CSS_PM_D2225_20240421_143306.dat',
 'FS1_CSS_PM_D2225_20240421_140907.dat',
 'FS1_CSS_PM_D2225_20240421_142247.dat',
 'FS1_CSS_PM_D2225_20240421_141116.dat',
 'FS1_CSS_PM_D2225_20240421_142659.dat',
 'FS1_CSS_PM_D2225_20240421_141325.dat']

In [25]:
raw= pp.read_raw_data_from_directory(os.path.join(ISAAC_database_path,ISAAC_data_rel_path))
for rel_path in ih.get_related_ISAAC_data_rel_path(ISAAC_data_rel_path,ISAAC_database_path,within_minutes=100):
    raw.update(pp.read_raw_data_from_directory(os.path.join(ISAAC_database_path,rel_path)))
raw_data_flist2 = list(raw.keys())
raw_data_flist2

['FS1_CSS_PM_D2225_20240421_140238.dat',
 'FS1_CSS_PM_D2225_20240421_140447.dat',
 'FS1_CSS_PM_D2225_20240421_140656.dat',
 'FS1_CSS_PM_D2225_20240421_140907.dat',
 'FS1_CSS_PM_D2225_20240421_141116.dat',
 'FS1_CSS_PM_D2225_20240421_141325.dat',
 'FS1_CSS_PM_D2225_20240421_141827.dat',
 'FS1_CSS_PM_D2225_20240421_142038.dat',
 'FS1_CSS_PM_D2225_20240421_142247.dat',
 'FS1_CSS_PM_D2225_20240421_142659.dat',
 'FS1_CSS_PM_D2225_20240421_143306.dat',
 'FS1_CSS_PM_D2225_20240421_144741.dat',
 'FS1_CSS_PM_D2225_20240421_151710.dat',
 'FS1_CSS_PM_D2225_20240421_151919.dat',
 'FS1_CSS_PM_D2225_20240421_152128.dat',
 'FS1_CSS_PM_D2225_20240421_152339.dat',
 'FS1_CSS_PM_D2225_20240421_152548.dat',
 'FS1_CSS_PM_D2225_20240421_152756.dat',
 'FS1_CSS_PM_D2225_20240421_153036.dat',
 'FS1_CSS_PM_D2225_20240421_153515.dat',
 'FS1_CSS_PM_D2225_20240421_153725.dat',
 'FS1_CSS_PM_D2225_20240421_153933.dat']

In [26]:

for key, value in raw.items():
    r = 0.5 * value['wire_diam']
    value['postprocess'] = {'denoised': [],
                            'projected': {}}
    rms = []
    arr_rms = []
    for ic, coord in enumerate(value['coord'][1:]):
        x = value['lraw'][ic][0]
        y = value['lraw'][ic][1]
        with capture_warnings():
            tmp = sp.process_profile_signal(x, y, r)
        value['postprocess']['denoised'].append(tmp)
        rms.append(tmp['rms_deconv'])
        arr_rms.append(tmp['MC_stat']['arr_deconv_rms'])
    value.pop('lraw')

    with capture_warnings():
        x,y,cxy,u,v = sp.project_L3_to_xyuv(*rms,value)
    with capture_warnings():    
        arr_x,arr_y,arr_cxy,arr_u,arr_v = sp.project_L3_to_xyuv(*arr_rms,value)
    value['postprocess']['projected']['xrms']=x
    value['postprocess']['projected']['yrms']=y
    value['postprocess']['projected']['cxy']=cxy
    value['postprocess']['projected']['urms']=u
    value['postprocess']['projected']['vrms']=v
    value['postprocess']['projected']['xrms_err']=np.std(arr_x)
    value['postprocess']['projected']['yrms_err']=np.std(arr_y)
    value['postprocess']['projected']['cxy_err']=np.std(arr_cxy)
    value['postprocess']['projected']['u_err']=np.std(arr_u)
    value['postprocess']['projected']['v_err']=np.std(arr_v)

In [27]:
# f = 'FS1_CSS_PM_D2225_20240528_200148.dat'
# plt.figure(figsize=(4,2))
# plt.plot(raw[f]['postprocess']['denoised'][0]['smoothed'])

# Organize for flame reconst input

In [28]:
from FRIB_model import flame_helper as fh
fm_orig = fh.ModelFlame(isac_data['fmlatfile'])
fm = fh.ModelFlame(isac_data['fmlatfile'])

In [29]:
nscan = len(isac_data['reconst_input']['meas'])
select = isac_data['reconst_input']['reconst_input']['measurement_select'][-1]
assert nscan == len(raw_data_flist)

fm_evals = {}
fm_goals = {}
fm_goals_postprocessed = {}
fm_goals_err = {}

for i in range(nscan):
    for fm_elem,val_field in isac_data['reconst_input']['meas'][i]['flamevall'].items():
        k = (fm_elem,val_field[1])
        if k not in fm_evals:
            fm_evals[k] = [None]*nscan
        fm_evals[k][i] = val_field[0]
        
    for fm_elem,meas in isac_data['reconst_input']['meas'][i]['monitorl'].items():
        for goal in ['xrms','yrms','cxy']:
            k = (fm_elem,goal)
            if k not in fm_goals:
                fm_goals[k] = [None]*nscan
                fm_goals_postprocessed[k] = [None]*nscan
                fm_goals_err[k] = [None]*nscan
            # if selected, record goal
            if select[goal][i]: 
                fm_goals[k][i] = meas[goal]
                fm_goals_postprocessed[k][i] = raw[raw_data_flist[i]]['postprocess']['projected'][goal]
                fm_goals_err[k][i] = raw[raw_data_flist[i]]['postprocess']['projected'][goal+'_err']
                
fm_evals = fh.make_FLAME_evals_or_goals(fm,df=pd.DataFrame(fm_evals))
fm_goals = fh.make_FLAME_evals_or_goals(fm,df=pd.DataFrame(fm_goals))
fm_goals_postprocessed = fh.make_FLAME_evals_or_goals(fm,df=pd.DataFrame(fm_goals_postprocessed))

fm_goals_err = pd.DataFrame(fm_goals_err)
scaler = np.mean(fm_goals_postprocessed['df'].std()/fm_goals_err.mean())
# fm_goals['normalization_factor'] = fm_goals_err
fm_goals_postprocessed['normalization_factor'] = fm_goals_err*scaler  # make loss roughly order of 1

In [30]:
fm_evals

{'info': {'FS1_CSS:QH_D2194': {'index': 100},
  'FS1_CSS:QV_D2202': {'index': 104}},
 'df':    FS1_CSS:QH_D2194 FS1_CSS:QV_D2202
                  B2               B2
 0           7.79765        -12.36668
 1           8.76214        -12.36668
 2           9.81842        -12.36574
 3          10.65401        -12.36668
 4          11.83920        -12.36574
 5          12.79515        -12.36668
 6           5.77308        -10.82952
 7           5.77213        -11.60806
 8           5.77308        -12.38281
 9           5.77308        -13.03506
 10          5.77308        -13.93231}

In [31]:
fm_goals

{'info': {'FS1_CSS:PM_D2225': {'index': 117}},
 'df':    FS1_CSS:PM_D2225                  
                xrms     yrms      cxy
 0           1.92800      NaN -0.06072
 1           0.92031      NaN -0.02683
 2           0.47139      NaN  0.38515
 3           1.35075      NaN  0.36085
 4               NaN      NaN  0.29826
 5               NaN      NaN  0.34669
 6               NaN  2.50952 -0.10732
 7               NaN  1.37676 -0.13447
 8               NaN  0.37788 -0.16403
 9               NaN  0.98336      NaN
 10              NaN  2.30465      NaN}

In [37]:
fm_goals_postprocessed['df']

FS1_CSS:PM_D2225                    
               xrms      yrms       cxy
0          1.927913       NaN -0.069275
1          0.916559       NaN -0.040085
2          0.448055       NaN  0.407908
3          1.350790       NaN  0.242821
4               NaN       NaN  0.291118
5               NaN       NaN  0.375129
6               NaN  2.500396 -0.104585
7               NaN  1.381674 -0.137782
8               NaN  0.376186 -0.170990
9               NaN  0.960846       NaN
10              NaN  2.313059       NaN

In [33]:
fm_goals_err

FS1_CSS:PM_D2225                    
               xrms      yrms       cxy
0          0.001338       NaN  0.005941
1          0.001542       NaN  0.009108
2          0.001026       NaN  0.039506
3          0.000813       NaN  0.025690
4               NaN       NaN  0.030783
5               NaN       NaN  0.026850
6               NaN  0.018809  0.005324
7               NaN  0.005051  0.003491
8               NaN  0.001457  0.013593
9               NaN  0.003597       NaN
10              NaN  0.012611       NaN

### compare fittings

In [34]:
fh.fit_moment1(fm,fm_evals,fm_goals_postprocessed,stop_criteria=0.1)
fm.bmstate.get_twiss('x'), fm.bmstate.xemittance, fm.bmstate.get_twiss('y'), fm.bmstate.yemittance

0-th trial, current_loss: 0.00257, best_loss: 0.00257
2-th trial, current_loss: 0.00257, best_loss: 0.00257


(array([0.21389025, 3.18141016, 0.32870614]),
 0.7781032870254277,
 array([-0.42528906,  2.78674382,  0.42374573]),
 0.85579517080842)

In [35]:
fh.fit_moment1(fm,fm_evals,fm_goals)
fm.bmstate.get_twiss('x'), fm.bmstate.xnemittance, fm.bmstate.get_twiss('y'), fm.bmstate.ynemittance

0-th trial, current_loss: 0.00195, best_loss: 0.00195
2-th trial, current_loss: 0.00155, best_loss: 0.00155


(array([0.16812521, 4.07654947, 0.25223932]),
 0.18880013247678654,
 array([-0.41415798,  2.94427371,  0.39790011]),
 0.16545374164908216)

In [36]:
fm_orig.bmstate.get_twiss('x'), fm_orig.bmstate.xnemittance, fm_orig.bmstate.get_twiss('y'), fm_orig.bmstate.ynemittance

(array([0.22605875, 6.07082222, 0.17314007]),
 0.24553420844292886,
 array([0.05119952, 3.01791607, 0.33222309]),
 0.24585141418812814)

from PMpostprocess.PMpostprocess import *

def estimate_two_noise_model(y,smoothed):
    scaler = np.max(smoothed)-np.min(smoothed)
    
    y = y/scaler
    smoothed = smoothed/scaler
    
    istart, iend = get_istart_iend_profile(smoothed)
    var_ysmooth = np.var(smoothed)
    var_noise = np.var(y-smoothed)
    
    noise_floor_offset, noise_floor_std = estimate_noise_floor(y,istart,iend)
    y = cut_boundary(y,istart,iend,noise_floor_offset)
    noise_signal_std = np.std(y[istart+1:iend-1]/smoothed[istart+1:iend-1])
    
    def loss(params):
        sigma1, sigma2 = params
        estimated_var_noise = sigma1**2 + sigma2**2 * var_ysmooth
        regularization = (sigma1/noise_floor_std-1)**2 + (sigma2/noise_signal_std-1)**2
        return 0.1*(estimated_var_noise/var_noise-1)**2 + 0.9*regularization

    result = minimize(loss, [noise_floor_std,noise_signal_std], bounds=[(0, 0.2), (0, 0.2)])
    noise_floor_std,noise_signal_std = result.x[0]*scaler,result.x[1]
    
    return noise_floor_std,noise_signal_std

def smooth_n_wire_deconvolve(x,y,r):#,finetune_deconv=False):
    """
    x : position array
    y : signal array
    r : wire radius
    """    
    xu, smoothed = gaussian_smooth_with_deconvolution(x, y,is_x_uniform=False)
    yu = interp1d(x, y)(xu)
    n = len(xu)
    if r is not None and r > 0:
        xu, wire_deconvolved = wire_deconvolution(xu,smoothed,r,is_x_uniform=True)
    else:
        wire_deconvolved = smoothed
    
    istart, iend = get_istart_iend_profile(smoothed)
    noise_offset, _ = estimate_noise_floor(yu,istart,iend)
    yu = cut_boundary(yu,istart,iend,noise_offset)
    
    rms_beam_size   = measure_rms_size(xu, yu)
    rms_smooth_size = measure_rms_size(xu, smoothed)
    rms_deconv_size = measure_rms_size(xu, wire_deconvolved)
    
    smoothed = interp1d(xu,smoothed)(x)
    wire_deconvolved = interp1d(xu,wire_deconvolved)(x)
    
    return smoothed, wire_deconvolved, rms_beam_size, rms_smooth_size, rms_deconv_size
    
    
def rms_uncertainty_quantification(x,smoothed, noise_floor_std, noise_signal_std, r=None, nMC=100):
    arr_deconv_rms = np.zeros(nMC)
    arr_smooth_rms = np.zeros(nMC)
    arr_noisy_rms = np.zeros(nMC)
    arr_y_samples = np.zeros((nMC,len(x)))
    for i in range(nMC):
        y_ = smoothed*(1+noise_signal_std*np.random.randn(*x.shape)) + noise_floor_std*np.random.randn(*x.shape)
        _,_, noisy_rms, smooth_rms, deconv_rms = smooth_n_wire_deconvolve(x,y_,r)
        arr_smooth_rms[i] = smooth_rms
        arr_deconv_rms[i] = deconv_rms
        arr_noisy_rms[i] = noisy_rms
        arr_y_samples[i] = y_

    # Calculating statistics
    smooth_mean, smooth_std = np.nanmean(arr_smooth_rms), np.nanstd(arr_smooth_rms)
    deconv_mean, deconv_std = np.nanmean(arr_deconv_rms), np.nanstd(arr_deconv_rms)
    noisy_mean , noisy_std  = np.nanmean(arr_noisy_rms), np.nanstd(arr_noisy_rms)

    return {
            "smooth_mean": smooth_mean,
            "smooth_std": smooth_std,
            "deconv_mean": deconv_mean,
            "deconv_std": deconv_std,
            "noisy_mean": noisy_mean,
            "noisy_std": noisy_std,
            "y_samples": arr_y_samples,
            }
    
def process_raw_profile(x,y,r):
    smoothed, wire_deconvolved, rms_noisy, rms_smooth, rms_deconv = smooth_n_wire_deconvolve(x,y,r)
    noise_floor_std,noise_signal_std = estimate_two_noise_model(y,smoothed)
    stat = rms_uncertainty_quantification(x, smoothed, noise_floor_std, noise_signal_std, r, nMC=100)
    
#     fig, axes = plt.subplots(1, 2, figsize=(14, 5), dpi=96)  # Create a figure with two subplots in a row
#     axes[0].plot(x, y, color="black", label=f"Noisy Profile,  $\\sigma_x$={rms_noisy:.3f} $\\pm$ {stat['noisy_std']:.3f} mm")
#     axes[0].plot(x, smoothed, color="green", label=f"Smoothed,   $\\sigma_x$={rms_smooth:.3f} $\\pm$ {stat['smooth_std']:.3f} mm")
#     axes[0].plot(x, wire_deconvolved, '--', color="red", label=f"DeConvolved, $\\sigma_x$={rms_deconv:.3f} $\\pm$ {stat['deconv_std']:.3f} mm")
#     axes[0].legend()
#     axes[0].set_xlabel("Position (mm)")
#     axes[0].set_ylabel("Signal Strength")
#     axes[0].set_ylim(0, 1.4 * max(y))
#     axes[0].set_title(f"Wire-thickness: {r:.3f} $mm$")

#     axes[1].plot(x, y - smoothed, 'k', label="Noisy - Smoothed")
#     for i in range(4):
#         y_ = stat['y_samples'][i]
#         axes[1].plot(x, y_ - smoothed, ':', label=f"Sample {i+1}")
#     axes[1].set_xlabel("Position (mm)")
#     axes[1].set_ylabel("Difference")
#     axes[1].legend()
#     axes[1].set_title("Differences with Smoothed Profile")
#     # Adjust layout for better spacing
#     plt.tight_layout()
